## 1. Introducción
Analizaremos  de nuevo el experimento de **bono para el reempleo** de Pensilvania, PERO SOBRE UN GRUPO DE TRATAMIENTO DISTINTO. Este fue llevado acabo en la década de 1980 por el Departamento de Trabajo de los Estados Unidos con el objetivo de evaluar cuál es el efecto de incentivar a las personas desempleadas con diferentes esquemas de compensación para acelerar su búsqueda de trabajo y con ello  disminuir las solicitudes al seguro de desempleo.

## 2. Diseño del experimento
Durante la fase inicial se construyeron diferentes grupos, los cuales fueron seleccionados de forma aleatoria:

- **Grupo Control:** Personas deseempleadas que reclamaron el seguro y que recibieron un trato tradicional sin incentivos.
- **Grupo experimental:** Personas desempleadas, que reclamaron el seguro, y que se les ofreció un incentivo económico (bono) en caso de encontrar un empleo en un rango de tiempo específico (periodo); siempre y cuando también se mantuviera el empleo durante una duración específica.

El grupo experimental está conformado por varios subgrupos, los cuales se diferenciaron en 3 aspectos clave:
- Monto del bono ofrecido.
- Duración del periodo para encontrar empleo.
- Si el bono disminuía o no progresivamente, dependiendo del período que tomaba encontrar empleo.
- Si se le ofrece o no un taller para encontrar empleo.

Lo anterior construye un total de 6 subgrupos experimentales que pueden contrastarse con el grupo control.

En esta ocasión **nos concentraremos sólo en el grupo experimental "T2"(treatment 2)**, el cual comparado con el grupo control, podemos resumirlo en la siguiente tabla:

| Grupo           | Monto del bono    | Tiempo de búsqueda (periodo) | ¿Se impartió un taller? |
|------------------|-----------------|-----------------------|----------------|
| 0 (Grupo Control)| 0 (No aplica)               | 0 (No aplica)                     | No             |
| 2 (Grupo Experimental)      | Bajo           | Largo                  | Si            |

Los datos los encuentras en el csv adjunto de nombre **"experiment.csv".**

### El objetivo es estimar el efecto del tratamiento en la duración del desempleo

### 2.1 Lectura y entendimiento de la información

In [ ]:
# Importamos las librerias necesarias
import pandas as pd
import os
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
import numpy as np
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [ ]:
# Cargamos la información de la fuente de datos del experimento
Penn = pd.read_csv('experiment.csv',sep=',')

# Filtramos y nos quedamos con los grupos que nos interesan, donde:
# Control: 0
# Experimental: 2
Penn = Penn[ (Penn['tg'] == 2) | (Penn['tg'] == 0) ]

# Transformamos nuestra variable del grupo experimental al target
Penn['T2'] = (Penn[['tg']]==2).astype(int)

# Creamos variable categorica para dep
Penn['dep'] = Penn['dep'].astype( 'category' )
Penn.head()


In [ ]:
# Columnas de la tabla
Penn.columns

La tabla contiene 23 variables (columnas), las cuales se describen de la siguiente manera:

- **abdt:** Representa el tiempo cronológico de inscripción de cada participante en el experimento.

- **tg:** Indica el grupo experimental al que pertenece cada participante.
  - Si tg = 0, el participante está en el grupo de control.
  - Si tg = 1, el participante está en el grupo 1, y así sucesivamente.

- **inuidur1:** Mide la duración (en semanas) del primer período de desempleo.

- **inuidur2:** Otra medida de la duración (en semanas) del primer período de desempleo. Esta variable no fue utilizada en el análisis.

- **female:** Variable dummy que indica el sexo del participante. Toma el valor de 1 si el participante es mujer, y 0 si es hombre.

- **black:** Variable dummy que indica si el participante es de raza negra (1 = sí, 0 = no).

- **hispanic:** Variable dummy que indica si el participante es de origen hispano (1 = sí, 0 = no).

- **othrace:** Variable dummy que indica si el participante pertenece a otra raza (no blanca, no negra y no hispana; 1 = sí, 0 = no).

- **dep:** Número de dependientes de cada participante.
  - Si el participante tiene 2 o más dependientes, toma el valor de 2.
  - De lo contrario, será 0 o 1, según corresponda.

- **q1 - q6:** Seis variables dummy que indican el trimestre del experimento en el cual se inscribió cada participante.

- **recall:** Toma el valor de 1 si el participante respondió "sí" cuando se le preguntó si esperaba ser reincorporado a su empleo anterior.

- **agelt35:** Toma el valor de 1 si la edad del participante es menor a 35 años, y 0 en caso contrario.

- **agegt54:** Toma el valor de 1 si la edad del participante es mayor a 54 años, y 0 en caso contrario.

- **durable:** Toma el valor de 1 si la ocupación del participante se encontraba en el sector de manufactura de bienes duraderos, y 0 en caso contrario.

- **nondurable:** Toma el valor de 1 si la ocupación del participante se encontraba en el sector de manufactura de bienes no duraderos, y 0 en caso contrario.

- **lusd:** Toma el valor de 1 si el participante presentó su reclamo en las ciudades de Coatesville, Reading o Lancaster, y 0 en caso contrario. Estas áreas se caracterizan por tasas bajas de desempleo y períodos cortos de desempleo.

- **husd:** Toma el valor de 1 si el participante presentó su reclamo en las ciudades deLewistown, Pittston o Scranton, y 0 en caso contrario. Estas áreas se caracterizan por tasas altas de desempleo y períodos cortos de desempleo.

- **muld:** Toma el valor de 1 si el participante presentó su reclamo en las ciudades de Philadelphia-North, Philadelphia-Uptown, McKeesport, Erie o Butler, y 0 en caso contrario. Estas áreas se caracterizan por tasas moderadas de desempleo y períodos largos de desempleo.

En nuestro análisis consideraremos modelos de regresión de la forma:
$$
Y = D \alpha_1 + W \beta + \epsilon
$$

Donde:

- $Y$ : Es el **logaritmo natural** de la duración del desempleo (*inuidur1*). Se aplica para estandarizar los tiempos de desemplo e interpretar mejor los resultados y comparaciones. 
- $D$: Es un indicador de tratamiento, que señala si un individuo pertenece a un grupo de tratamiento específico. En nuestro caso seria 1 (variable T2) si una persona pertenece al grupo experimental y 0 en caso de no ser asi.
-$W$: Es el vector de variables restantes que incluye:
  - Variables dummy para los grupos de edad.
  - Género.
  - Raza.
  - Número de dependientes.
  - Trimestre del experimento.
  - Localización dentro del estado.
  - Existencia de expectativas de reincorporación al empleo.
  - Tipo de ocupación.
- $\alpha_1$: No confundir con un coeficiente de la regresion común, este representa el efecto promedio de aplicar el tratamiento a la población experimental.
- $\beta$: Son  todos los demás coeficientes de regresión, es decir, los pesos que se le asigna a cada variable en funcion de $Y$
- ϵ: Es el término de error aleatorio.


### 3. Análisis

##### Usa la función LogisticRegression(), como vimos en clase, para ajustar una regresión logística y ver si la asignación al tratamiento "depende" de las covariables. Para ello considera la siguiente formulación de predictoras y variable respuesta:

In [ ]:
X_logit = patsy.dmatrix("""(female + black + othrace + C(dep) + q2 + q3 + q4 +
                      q5 + q6 + agelt35 + agegt54 + durable + lusd + husd)**2 - 1""",
                  data=Penn, return_type='dataframe')
y_logit = Penn["T2"]


- a) ¿Cuál es el valor del área bajo la curva ROC (AUC) del modelo de regresión logística? ¿Qué puedes decir acerca de este valor específico?
- b) Grafica la distribución de la "propensity score" en ambos grupos: control y tratamiento. ¿Qué observas?
- c) Ajusta un modelo "simple" de la forma
$$
Y = D \alpha_1 + \epsilon
$$
- d) ¿Cuál es el valor estimado del efecto causal en este modelo y cómo lo interpretas?
- e) ¿Consideras pertinente ajustar un modelo incluyendo las covariables W? Justifica tu respuesta. En caso afirmativo, ejecuta el modelo:
$$
Y = D \alpha_1 + W \beta + \epsilon
$$

- f) ¿Cuál es el valor estimado del efecto causal en este modelo y cómo lo interpretas?
- g) Explica a qué se debe la diferencia entre la magnitud del efecto causal estimado en d) vs el estimado en f).
- h) ¿Qué diagnósticos adicionales propondrías para asegurar la confiabilidad de tus estimaciones?

In [1]:
# Nota: puedes pasarla a ols las siguientes fórmulas en el parámetro formula:
# formula_lr = "np.log(inuidur1) ~ T2"
# formula_lr = """np.log(inuidur1) ~ T2 + female + black + othrace + C(dep) +
#                  q2 + q3 + q4 + q5 + q6 + agelt35 + agegt54 + durable + lusd + husd"""